1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
   - 토큰수 초과로 답변을 생성하지 못할 수 있고
   - 문서가 길면(인풋이 길면) 답변 생성이 오래 걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 백터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

In [ ]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters
%pip install --quiet langchain-chroma
%pip install -U langchain langchainhub --quiet

In [46]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter = text_splitter)

len(document_list)

225

In [47]:
from langchain_ollama import OllamaEmbeddings
from dotenv import load_dotenv
import os
load_dotenv()


embeddings = OllamaEmbeddings(model=os.getenv("embedmodel"))

In [49]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(
#     documents = document_list,
#     embedding = embeddings,
#     persist_directory = "./chroma_db",
#     collection_name = "tax_collection",
# )

database = Chroma(persist_directory="./chroma_db", collection_name="tax_collection", embedding_function=embeddings)


In [50]:
query = '연봉 7천만원인 직장인의 소득세는 얼마인가요?'
# retrieved_docs = database.similarity_search(query, k=3)

In [33]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model=os.getenv("model"))

In [ ]:
# prompt= f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# Question: {query}
# """

In [ ]:
# ai_message = llm.invoke(prompt)

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

In [53]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type_kwargs = {"prompt": prompt},
    retriever = database.as_retriever(),
)



In [54]:
ai_message = qa_chain.invoke({"query": query})

In [ ]:
ai_message